In [ ]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git 

In [4]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import time
import os
from pykospacing import Spacing

PASSPORT_KEY = "831f995b3bce1814e5af8034c26fda874821af0b"

BASE_URL = "https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy"
spacing = Spacing()

def remove_tags(text: str) -> str:
    text = u'<content>{}</content>'.format(text).replace('<br>', '')
    return ''.join(ET.fromstring(text).itertext())

def spell_check_naver(text: str) -> tuple:
    params = {"passportKey": PASSPORT_KEY, "q": text, "color_blindness": "0"}
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://search.naver.com/"
    }
    try:
        response = requests.get(BASE_URL, params=params, headers=headers)
        data = response.json()['message']['result']
        corrected = remove_tags(data.get('html', ''))
        return corrected, data.get('errata_count', 0)
    except Exception as e:
        print(f"❌ 오류: {e}")
        return text, -1

def process_in_batches(input_path: str, batch_size: int = 100, text_column: str = "text"):
    df = pd.read_csv(input_path, encoding='utf-8')
    total = len(df)
    output_path = "output_partial.csv"

    # 기존 파일 삭제 (처음 실행 시)
    # if os.path.exists(output_path):
    #     os.remove(output_path)
    start_idx = 1
    for start in range(start_idx, total, batch_size):
        end = min(start + batch_size, total)
        batch_df = df.iloc[start:end].copy()

        corrected_list = []
        error_count_list = []

        for text in batch_df[text_column]:
            text = str(text)
            corrected, error_count = spell_check_naver(text)
            corrected_list.append(corrected)
            error_count_list.append(error_count)

            time.sleep(1)

        batch_df['sentence'] = corrected_list  # 맞춤법 교정 결과
        batch_df['count'] = error_count_list  # 맞춤법 오류 수

        batch_df.to_csv(output_path, mode='a', index=False, header=not os.path.exists(output_path), encoding='utf-8-sig')
        print(f"✅ {start}~{end} 저장 완료")

    # 최종 결과 저장
    final_df = pd.read_csv(output_path, encoding='utf-8-sig')
    final_df.to_csv("output_final.csv", index=False, encoding='utf-8-sig')
    print("🎉 최종 저장 완료: output_final.csv")


In [5]:
process_in_batches("n_duple_reviews.csv")

✅ 1~101 저장 완료
✅ 101~201 저장 완료
✅ 201~301 저장 완료
✅ 301~401 저장 완료
✅ 401~501 저장 완료
✅ 501~601 저장 완료
✅ 601~701 저장 완료
❌ 오류: Expecting value: line 1 column 1 (char 0)
✅ 701~801 저장 완료
✅ 801~901 저장 완료
❌ 오류: Expecting value: line 1 column 1 (char 0)
❌ 오류: Expecting value: line 1 column 1 (char 0)
✅ 901~1001 저장 완료
✅ 1001~1101 저장 완료
✅ 1101~1201 저장 완료
✅ 1201~1301 저장 완료
✅ 1301~1401 저장 완료
✅ 1401~1501 저장 완료
✅ 1501~1601 저장 완료
✅ 1601~1701 저장 완료
✅ 1701~1801 저장 완료
✅ 1801~1901 저장 완료
✅ 1901~2001 저장 완료
✅ 2001~2101 저장 완료
✅ 2101~2201 저장 완료
✅ 2201~2301 저장 완료
❌ 오류: Expecting value: line 1 column 1 (char 0)
❌ 오류: Expecting value: line 1 column 1 (char 0)
❌ 오류: Expecting value: line 1 column 1 (char 0)
❌ 오류: Expecting value: line 1 column 1 (char 0)
❌ 오류: Expecting value: line 1 column 1 (char 0)
✅ 2301~2401 저장 완료
❌ 오류: Expecting value: line 1 column 1 (char 0)
❌ 오류: Expecting value: line 1 column 1 (char 0)
✅ 2401~2501 저장 완료
✅ 2501~2601 저장 완료
✅ 2601~2701 저장 완료
❌ 오류: Expecting value: line 1 column 1 (char 0)
❌ 오류: E

In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import time
from pykospacing import Spacing

# 네이버 맞춤법 검사 js에 보낼 키와 주소
PASSPORT_KEY = "dbc0dc1e7ae1070bacecf1a3865c72b5d121abc2"
BASE_URL = "https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy"

spacing = Spacing()

def remove_tags(text: str) -> str:
    text = u'<content>{}</content>'.format(text).replace('<br>', '')
    return ''.join(ET.fromstring(text).itertext())

def spell_check_naver(text: str) -> tuple:
    params = {
        "passportKey": PASSPORT_KEY,
        "q": text,
        "color_blindness": "0"
    }
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://search.naver.com/"
    }
    try:
        response = requests.get(BASE_URL, params=params, headers=headers)
        data = response.json()['message']['result']
        html = data.get('html', '')
        corrected = remove_tags(html)
        return corrected, data.get('errata_count', 0)
    except Exception as e:
        print(f"❌ 오류: {e}")
        return text, -1

def extract_diff_words(original: str, corrected: str) -> str:
    ori_words = original.split()
    cor_words = corrected.split()
    diffs = []

    for o, c in zip(ori_words, cor_words):
        if o != c:
            diffs.append(f"{o}→{c}")

    if len(ori_words) != len(cor_words):
        min_len = min(len(ori_words), len(cor_words))
        tail_ori = ori_words[min_len:]
        tail_cor = cor_words[min_len:]
        for o, c in zip(tail_ori, tail_cor):
            if o != c:
                diffs.append(f"{o}→{c}")

    return ", ".join(diffs)

def process_and_add_columns(input_path: str, output_path: str, text_column: str = "review"):
    df = pd.read_csv(input_path, encoding='utf-8')

    corrected_list = []
    error_count_list = []
    diff_list = []

    for idx, row in df.iterrows():
        text = str(row[text_column])
        
        # 2. 네이버 맞춤법 검사
        corrected, error_count = spell_check_naver(text)

        # 3. 차이 추출
        diff = extract_diff_words(text_spaced, corrected) if error_count > 0 else ""
        text_spaced = spacing(corrected)
        corrected_list.append(corrected)
        error_count_list.append(error_count)
        diff_list.append(diff)
        
        time.sleep(0.8)

    df['spacing_fixed'] = [spacing(str(x)) for x in df[text_column]]  # 띄어쓰기 자동 보정 컬럼도 추가
    df['완성'] = text_spaced
    df['corrected_review'] = corrected_list
    df['spell_error_count'] = error_count_list
    df['spell_diff'] = diff_list

    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\n✅ 결과 저장 완료: {output_path}")


In [ ]:
corrected_list = []
error_count_list = []
diff_list = []

In [ ]:
inputPath = 'random_reviews.csv'
outputfile = 'result_random.reviews.csv'
process_and_add_columns(inputPath, outputfile, 'text')

time.sleep(0.8)
# df.to_csv('random_.csv', encoding='utf-8-sig', header=True)

In [ ]:
input_file = "dict_review_texts.csv"
output_file = "리뷰_띄어쓰기_맞춤법_교정.csv"

process_and_add_columns(input_file, output_file)